In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import defaultdict
from collections import Counter
from src.classes.database.offer import MongoOffer
from tqdm import tqdm

tokenizer = nltk.RegexpTokenizer(r'\w+')

In [2]:
token_list = [tokenizer.tokenize(x.selling) for x in MongoOffer.objects()][0:100]
tagged = nltk.pos_tag(token_list[0])

In [3]:
def get_all_words(token_list: list[list]) -> list:
    words = set()
    for sentence in token_list:
        for word in sentence:
            words.add(word)
    return sorted(words)

def next_word_occurrence(key: str, look_ahead: int, tokens: list[list]) -> dict:
    counter = defaultdict(int)
    for sentence in tokens:
        for x in range(0, len(sentence)):
            if key == sentence[x]: #if we find the key in a sentence
                if (x+look_ahead) < len(sentence):
                    counter[sentence[x+look_ahead]] += 1
    return counter

def p_next_word(given: str, looking: str, look_ahead: int, tokens: list[list]) -> float:
    nwo: dict = next_word_occurrence(given, look_ahead, tokens)
    return nwo[looking] / sum(nwo.values()) if nwo[looking] != 0 else 0

def calculate_all_probabilities(tokens: list[list]) -> dict:
    all_words = get_all_words(tokens)

    #Generating a list of all words
    word_canvas = {}
    for word in all_words:
        word_canvas[word] = None

    p_word = word_canvas.copy()
    for word in tqdm(p_word):
        p_word[word] = [word_canvas.copy() for x in range(0,5)]
        for x in range(0,5):
            for sub_word in p_word[word][x].keys():
                p_word[word][x][sub_word] = p_next_word(word, sub_word, x+1, tokens)
                #print(f"{word} | {str(x)} | {sub_word}: {p_word[word][x][sub_word]}")


    return p_word

In [4]:
data = calculate_all_probabilities(token_list)

100%|██████████| 398/398 [00:18<00:00, 21.99it/s]


In [103]:
sum(data['i5'][0].values())

1.0

In [ ]:
class Sentence:
    def __init__(self):


class Word:
    def __init__(self, word: str):
        self.word = word
        self.p_of_next_word = [next_word_occurrence(word, x, token_list) for x in range(1,6)]


In [ ]:
test = Word('i5')

In [ ]:
test.p_of_next_word[1]

p_of_next_word['word']['looking_for_word'][INDEX HERE OF LOOKAHEAD]
return object like {occurrences: 1, probability: 0.5}

word1
    looking_for1
        0: {occurenecs:1, probability: 0.5}
        1:  {occurenecs:1, probability: 0.5}
        2:  {occurenecs:0, probability: 5}
    looking_for2